In [4]:
import rebound
import reboundx
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import pandas as pd

sys.path.append('../')

from ic import output, initialize, plotsa

from collections import OrderedDict
planets = ['b', 'c', 'd', 'e', 'f', 'g', 'h']
resonances = OrderedDict([(('b','c'),(8,5)),(('c','d'),(5,3)),(('d','e'),(3,2)),(('e','f'),(3,2)),(('f','g'),(4,3)),(('g','h'),(3,2))]) # ordered so we add planets in right sequence
threebodyresonances = OrderedDict([(('b','c','d'),(2,3)),(('c','d','e'),(1,2)),(('d','e','f'),(2,3)),(('e','f','g'),(1,2)),(('f','g','h'),(1,1))])

In [5]:
def plotscaling(mag, taues):
    fig, axarr = plt.subplots(ncols=1, nrows=4, figsize=(18,10))
    colors = ['red','blue','green','cyan','magenta','yellow','black']
    handles = []

    for i, taue in enumerate(taues):
        filename='data/mag{0:.4e}taue{1:.2e}.bin'.format(mag,taue)
        sa = rebound.SimulationArchive(filename)

        color = colors[i]

        Nout = len(sa)
        N = sa[0].N

        outputs = initialize(planets, resonances, threebodyresonances)
        E = np.zeros(Nout)
        Lz = np.zeros(Nout)
        for i,sim in enumerate(sa):
            ps = sim.particles
            output(sim,planets,resonances,threebodyresonances,outputs)
            E[i] = sim.calculate_energy()
            x,y,Lz[i] = sim.calculate_angular_momentum()

        t, e, P, Pratio, phi1, phi2, deltapomega, phi3body = outputs
        compressionfactor = 1.e6*(taue/100)
        for key in t.keys():
            t[key] = [time/compressionfactor for time in t[key]]

        for p in planets:
            axarr[0].plot(t[p], e[p], '.', color=color, alpha=0.1)

        for resonance in resonances.items():
            pair = resonance[0]
            res = resonance[1]
            resratio = res[0]/res[1] 
            axarr[1].plot(t[pair], np.array(Pratio[pair])-resratio, '.', color=color, alpha=0.1)
            
        axarr[2].plot(t['b'], E, '.', color=color, alpha=0.1)
        axarr[3].plot(t['b'], Lz, '.', color=color, alpha=0.1)
        #for resonance in threebodyresonances.items():
        #    triad = resonance[0]
        #    axarr[1].plot(t[triad], phi3body[triad], '.', color=color, alpha=0.1)

        handles.append(mlines.Line2D([], [], color='white', marker='.',markersize=15, markerfacecolor=color, label="{0:.1e}".format(taue)))

    for ax in axarr.flatten():
        ax.legend(handles=handles, numpoints=1, loc='upper left')
    
    return fig

In [7]:
df = pd.read_csv("../data.csv", index_col=0)
df.head()

,K,mag,filename
0,125.210,0.139820,IC0K1.2521e+02mag1.3982e-01.bin
2,74.471,0.001196,IC2K7.4471e+01mag1.1961e-03.bin
4,859.130,0.043822,IC4K8.5913e+02mag4.3822e-02.bin
6,610.550,0.009907,IC6K6.1055e+02mag9.9069e-03.bin
7,14.211,0.218653,IC7K1.4211e+01mag2.1865e-01.bin


In [ ]:
for fname in df['filename']:
    sa = rebound.SimulationArchive(fname)
    